In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from dataloader.builder import build_dataset
from model.mlp import MLP
from uncertainty_estimator.masks import build_masks 
from experiment_setup import build_estimator
from analysis.metrics import get_uq_metrics

plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
config = {
    'random_seed': 43,
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    'k_folds': 10,
    'verbose': False,
    'use_cache': False,
    'layers': [13, 64, 32, 32, 1],
    'epochs': 10_000,
    'acc_percentile': 0.1,
    'patience': 3,
    'dropout_train': 0.2,
    'dropout_uq': 0.5,
    'batch_size': 32,
    'dataset': 'energy_efficiency',
    'scale': True,
    'l2_reg': 1e-5
}


In [ ]:
# Get data
dataset = build_dataset(config['dataset']) 
x_set, y_set = dataset.dataset('train')
x_test, y_test = dataset.dataset('val')
config['layers'][0] = x_set.shape[-1]

In [ ]:
# masks = build_masks(['vanilla', 'mirror_random', 'decorrelating', 'dpp'])
masks = build_masks(nn_runs=config['nn_runs'])

In [ ]:
# Helper functions
def scale(train, val):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    val = scaler.transform(val)
    return train, val, scaler


def plot_evaluations(model, x, y, x_, y_):
    plt.figure(figsize=(10, 10))
    plt.plot((min(y), max(y)), (min(y), max(y)))
    plt.scatter(model(x).cpu().numpy(), y)
    plt.scatter(model(x_).cpu().numpy(), y_)
    plt.show()


# Evaluate different masks
def evaluate_masks(model, masks, x_val, y_val):
    predictions = model(x_val).cpu().numpy()
    errors = np.abs(predictions - y_val)
    results = []

    for name, mask in masks.items():
        estimator = build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])

        for run in range(config['runs']):
            estimations = estimator.estimate(x_val)
            acc, ndcg, nll = get_uq_metrics(estimations, errors, config['acc_percentile'])
            results.append([acc, ndcg, nll, name])

            if hasattr(mask, 'reset'):
                mask.reset()

    return results


In [ ]:
def multiple_kfold(k, data_size, max_iterations):
    kfold = KFold(k)
    for i in range(max_iterations):
        if i % k == 0:
            data_idx = np.random.permutation(data_size)
            idx_generator = kfold.split(data_idx)
        train_idx, val_idx = next(idx_generator)
        yield data_idx[train_idx], data_idx[val_idx]

In [ ]:
layer_sizes = [
    [8, 64, 64, 32, 1],
    [8, 128, 128, 64, 1],
    [8, 256, 256, 128, 1],
    [8, 512, 512, 256, 1],
    [8, 1024, 1024, 512, 1]
]
# Train models and evaluate masks
config['epochs'] = 10_000 
config['max_runs'] = 20

mask_results = [] 

for layer_size in layer_sizes:
    config['layers'] = layer_size
    kfold_iterator = multiple_kfold(config['k_folds'], len(x_set), config['max_runs'])
    for m, (train_idx, val_idx) in enumerate(kfold_iterator):
        x_train, y_train = x_set[train_idx], y_set[train_idx]
        x_val, y_val = x_set[val_idx], y_set[val_idx]
        print(x_train.shape, y_train.shape, x_val.shape)
        print("Model", m+1)

        if config['scale']:
            x_train, x_val, _ = scale(x_train, x_val)
            y_train, y_val, y_scaler = scale(y_train, y_val)
        else:
            y_scaler = None


        model = MLP(config['layers'], l2_reg=config['l2_reg'])
        train_config = {k: config[k] for k in config if k in ['patience', 'dropout_rate', 'epochs', 'batch_size']}
        model.fit((x_train, y_train), (x_val, y_val), **train_config)
        if config['verbose']:
            plot_evaluations(model, x_train, y_train, x_val, y_val)
        results = evaluate_masks(model, masks, x_val, y_val)
        for result in results:
            result.append(layer_size[1])
        mask_results.extend(results)

In [ ]:
# Plot the results
mask_df = pd.DataFrame(mask_results, columns=['acc', 'ndcg', 'nll', 'mask', 'layer_size'])

plt.figure(figsize=(16, 18))

def boxplot(df, x_label, y_label, i, bottom=0, top=1):
    plt.subplot(3, 1, i)
    plt.xticks(rotation=45)
    if bottom is not None:
        plt.ylim(bottom, top)
    sns.boxplot(data=df, x=x_label, y=y_label, hue='layer_size')

boxplot(mask_df, 'mask', 'acc', 1, 0, 0.8)
boxplot(mask_df, 'mask', 'ndcg', 2, 0, 0.9)
boxplot(mask_df, 'mask', 'nll', 3, None)